1. Import libs and read excel (reading excel is slow)

In [2]:
from pprint import pprint
import pandas as pd
import re
data = pd.read_excel('../../Clarifications/G4 - mandatory data elements 11.10.2023 (1) - Copy.xlsx')


C:\Users\ciar\AppData\Local\Temp\ipykernel_25948\1252154880.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


2. Extract elements and sub-element names from clarifications excel sheet

In [7]:
print(data.keys())
clarification_elements = []

for i in range(data.shape[0]):
    element = data.at[i, 'Data element/class name ']
    if element != element:
        element = data.at[i, 'Data sub-element/sub-class name']
    clarification_elements.append(data.at[i, 'DE No. (Ref ID6)'] + " " + element.lower())

    
print(clarification_elements)


Index(['DE No. (Ref ID6)', 'Data element/class name ',
       'Data sub-element/sub-class name', 'Ref ID5', 'G4',
       'Should the data element (from column B) be filled out on the declaration? ',
       'If the data element (from column B) is being declared on the declaration - which data sub-elements (from column C) should then ALWAYS also be filled out? ',
       'Initial version', 'Comments'],
      dtype='object')
['11 03 000 000 goods item number', '11 03 001 000 goods item number', '11 04 000 000 specific circumstance indicator', '11 04 001 000 specific circumstance indicator', '12 01 000 000 previous document', '12 01 001 000 reference number', '12 01 002 000 type', '12 01 007 000 goods item identifier', '12 02 000 000 additional information', '12 02 008 000 code', '12 02 009 000 text', '12 03 000 000 supporting document', '12 03 001 000 reference number', '12 03 002 000 type', '12 04 000 000 additional reference', '12 04 002 000 type', '12 05 000 000 transport document', '12

3. Extract EUCDM name annotations from XSD file
(NOTE: this assumes all elements are correctly annotated, which for G4 they are because I manually edited them)

In [14]:

clarifications_filename = './G4_DMS_v0.1.xsd'

def extract_elements(text, search_by):
    # Define the regex pattern
    if search_by == 'Name':
        pattern = r"<EUCDMname>(.*?)</EUCDMname>"
    elif search_by == 'Number':
        pattern = r"<code>(.*?)</code>"
    elif search_by == 'Both':
        pattern = (
                r"<EUCDMname>(.*?)</EUCDMname>\s*"
                r"<EUCDMversion>(.*?)</EUCDMversion>\s*"
                r"<code>(.*?)</code>"
            )


    # Use re.findall to extract all occurrences
    matches = re.findall(pattern, text)
    # Return the matches
    return matches

text = open(clarifications_filename, 'r').read()
# element_name_annotations = extract_elements(text, 'Name')
# element_number_annotations = extract_elements(text, 'Number')
# print(len(element_number_annotations), len(element_name_annotations))
annotations = extract_elements(text, 'Both')
print(annotations)
#xxx


#schema_annotations = {element_name_annotations[i]: element_number_annotations[i] for i in range(len(element_name_annotations))} 
#schema_annotations = {annotations[i][0]: annotations[i][2] for i in range(len(annotations))} 

schema_annotations = []
for i in range(len(annotations)):
    schema_annotations.append( annotations[i][2] + " " + annotations[i][0].lower()) 

print(schema_annotations)





[('LRN', 'EUCDM 6.1', '12 09 001 000'), ('Specific circumstance indicator', 'EUCDM 6.1', '11 04 001 000'), ('Representative', 'EUCDM 6.1', '13 06 000 000'), ('Name', 'EUCDM 6.1', '13 06 016 000'), ('Identification number', 'EUCDM 6.1', '13 06 017 000'), ('Address', 'EUCDM 6.1', '13 03 018 000'), ('City', 'EUCDM 6.1', '13 03 018 022'), ('Country', 'EUCDM 6.1', '13 03 018 020'), ('Street additional line', 'EUCDM 6.1', '13 06 018 024'), ('Postcode', 'EUCDM 6.1', '13 06 018 021'), ('Street', 'EUCDM 6.1', '13 06 018 023'), ('Number', 'EUCDM 6.1', '13 06 018 025'), ('P.O. box', 'EUCDM 6.1', '13 06 018 026'), ('Sub-division', 'EUCDM 6.1', '13 06 018 027'), ('Communication', 'EUCDM 6.1', '13 06 029 000'), ('Identifier', 'EUCDM 6.1', '13 06 029 015'), ('Type', 'EUCDM 6.1', '13 06 029 002'), ('Status', 'EUCDM 6.1', '13 06 030 000'), ('Declarant', 'EUCDM 6.1', '13 05 000 000'), ('Name', 'EUCDM 6.1', '13 05 016 000'), ('Identification number', 'EUCDM 6.1', '13 05 017 000'), ('Address', 'EUCDM 6.1'

In [16]:
#schema_annotations =  {k.lower(): v for k, v in schema_annotations.items()}
print("The following elements from the clarifications excel sheet are missing from the XSD: ")
for item in clarification_elements:
    if item not in schema_annotations: #or val not in schema_annotations.values():
        print(item)
print("\n")

print("The following elements in the XSD are not present in the clarifications excel sheet: ")
for item in schema_annotations:
    if item not in clarification_elements: #or val not in schema_annotations.values():
        print(item)


The following elements from the clarifications excel sheet are missing from the XSD: 
11 04 000 000 specific circumstance indicator
12 09 000 000 lrn
12 11 002 000 type
13 03 029 000 communication
13 05 018 000 address
13 06 018 000 address
13 06 018 020 country
13 06 018 022 city
15 04 000 000 estimated date and time of arrival at the port of unloading
16 15 048 000 gnss
16 15 081 000 postcode address
16 15 081 021 postcode
17 10 000 000 supervising customs office
17 10 001 000 reference number
18 03 000 000 total gross mass
19 11 000 000 receptacle identification number


The following elements in the XSD are not present in the clarifications excel sheet: 
12 04 001 000 reference number
12 04 001 000 reference number
